In [27]:
import pandas as pd
import importlib
import time
import os
os.chdir("..")

In [45]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from nltk.tokenize import sent_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

In [24]:
%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv("../data/reviews_paris_hotels.csv")
data

,listing_name,listing_score,username,review_score,review_title,review
0,Hotel Moderniste,5.0,Alex M,5.0,Quality over the quantity!,l have stayed in many hotels. But Moderniste i...
1,Hotel Moderniste,5.0,Andrea Saade,5.0,Great experience!,We stayed at the Moderniste in December 2021 d...
2,Hotel Moderniste,5.0,halloCairns,5.0,Just renovated hotel in a nice neighbourhood i...,Very helpful at reception. Good service. Clean...
3,Hotel Malte - Astotel,5.0,Grace L,5.0,Fabulous,Myself & my husband cannot recommend this hote...
4,Hotel Malte - Astotel,5.0,TomosParsons,5.0,"Great location, modern and spacious","Great location, being so close to the Louvre a..."
...,...,...,...,...,...,...
208,St Christopher's Inn Canal Paris,4.0,Fernando T,5.0,Eurotrip,Amazing. The staff Pedro was very kind with us...
209,St Christopher's Inn Canal Paris,4.0,Spencer S,5.0,such an amazing trip,Barry is the sexiest bartender alive i love th...
210,St Christopher's Inn Canal Paris,4.0,Timmy B,4.0,Good value and good staff,"Overall a very good hostel experience, staff e..."
211,St Christopher's Inn Canal Paris,4.0,Teigan L,3.0,Pleasant Hostel,This was a pleasant stay. Very economical and ...


In [4]:
data["review"][0]

'l have stayed in many hotels. But Moderniste is really exceeded my expectation. Manager and all staff were very welcoming. Breakfast was excellent. Room was very nice and very comfortable bed.\n\nWe also love the location. On our next visit we will defiantly stay here.\n\nThank you Moderniste.'

In [32]:
from Preprocessors.ReviewPreprocessor import ReviewPreprocessor

In [30]:
import Preprocessors.ReviewPreprocessor

In [31]:
importlib.reload(Preprocessors.ReviewPreprocessor)

<module 'Preprocessors.ReviewPreprocessor' from 'E:\\PychCarmProjects\\s4\\SentiAnalysis\\Preprocessors\\ReviewPreprocessor.py'>

In [33]:
preprocessor = ReviewPreprocessor(data["review"])
t = time.time()
cleaned_reviews = preprocessor.start()
print(f"{time.time() - t} s")
cleaned_reviews

remove_tags done
spelling_correction done
remove_objective_sentences done
24.45242428779602 s


0      l have stayed in many hotels. Manager and all ...
1      We were a family of 4 (2 adults and 2 kids) an...
2      Good service. Clean. Nice room. The hotel is c...
3      Myself & my husband cannot recommend this hote...
4      Great location, being so close to the Louvre a...
                             ...                        
208    Amazing. The staff Pedro was very kind with us...
209    Barry is the sexiest bartender alive i love th...
210    Overall a very good hostel experience, staff e...
211    This was a pleasant stay. Very economical and ...
212    The room was clean and it seemed to be very ne...
Name: review, Length: 213, dtype: object

In [34]:
data["cleaned_review"] = cleaned_reviews

In [42]:
data["cleaned_review"][3]

'Myself & my husband cannot recommend this hotel highly enough. Rooms were beautiful and spacious, breakfast was lovely with plenty of different options to choose from.'

In [43]:
data["review"][3]

'Myself & my husband cannot recommend this hotel highly enough. From the minute we walked in the staff were all so helpful and welcoming. Rooms were beautiful and spacious, breakfast was lovely with plenty of different options to choose from. We will definitely be back.'

In [46]:
aspects = []
for id_, sentence in data["cleaned_review"].items():
    doc = nlp(sentence)
    descriptive_term = ''
    target = ''
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            target = token.text
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = prepend + token.text
    aspects.append({'aspect': target,
        'description': descriptive_term})
print(aspects)

[{'aspect': 'Room', 'description': 'very comfortable'}, {'aspect': 'holidays', 'description': 'enjoyable'}, {'aspect': 'staff', 'description': 'helpful'}, {'aspect': 'breakfast', 'description': 'different'}, {'aspect': 'hotel', 'description': 'great'}, {'aspect': 'Rooms', 'description': 'very comfortable'}, {'aspect': 'room', 'description': 'lovely'}, {'aspect': 'breakfast', 'description': 'special'}, {'aspect': 'choices', 'description': 'huge'}, {'aspect': 'staff', 'description': 'super helpful'}, {'aspect': 'hotel', 'description': 'soft'}, {'aspect': 'Staff', 'description': 'available'}, {'aspect': 'Breakfast', 'description': 'free'}, {'aspect': 'room', 'description': 'meticulously clean'}, {'aspect': 'place', 'description': 'impeccable'}, {'aspect': '', 'description': 'quiet'}, {'aspect': 'snacks', 'description': 'very welcome'}, {'aspect': 'nightlife', 'description': 'safe'}, {'aspect': 'staff', 'description': 'very kind'}, {'aspect': 'staff', 'description': 'very attentive'}, {'as